# Calcul de la var d'un portefeuille


In [111]:
# importer les packages
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import numpy as np 
import cufflinks as cf #

from random import random

In [23]:
cf.set_config_file(offline=True)

## 1)la méthode historique

In [3]:
# recupération du cour de nos actif

debut=dt.datetime(2019,1,1)
fin=dt.datetime(2021,1,1)
tickers=['TSLA', 'AAPL']
df=web.DataReader(tickers, 'yahoo', debut,fin)
df.head()

Attributes  Adj Close                 Close                  High             \
Symbols          TSLA       AAPL       TSLA       AAPL       TSLA       AAPL   
Date                                                                           
2019-01-02  62.023998  38.439735  62.023998  39.480000  63.026001  39.712502   
2019-01-03  60.071999  34.610851  60.071999  35.547501  61.880001  36.430000   
2019-01-04  63.537998  36.088364  63.537998  37.064999  63.599998  37.137501   
2019-01-07  66.991997  36.008041  66.991997  36.982498  67.348000  37.207500   
2019-01-08  67.070000  36.694466  67.070000  37.687500  68.802002  37.955002   

Attributes        Low                  Open                 Volume  \
Symbols          TSLA       AAPL       TSLA       AAPL        TSLA   
Date                                                                 
2019-01-02  59.759998  38.557499  61.220001  38.722500  58293000.0   
2019-01-03  59.476002  35.500000  61.400002  35.994999  34826000.0   
2019-01-04  60.546001  35.950001  61.200001  36.132500  36970500.0   
2019-01-07  63.549999  36.474998  64.344002  37.174999  37756000.0   
2019-01-08  65.403999  37.130001  68.391998  37.389999  35042500.0   

Attributes               
Symbols            AAPL  
Date                     
2019-01-02  148158800.0  
2019-01-03  365248800.0  
2019-01-04  234428400.0  
2019-01-07  219111200.0  
2019-01-08  164101200.0

In [7]:
#je cree une dataFrame qui contient les uniquement mes cours de cloture

actifs = df['Close']
actifs.head()

Symbols,TSLA,AAPL
Date,,
2019-01-02,62.023998,39.480000
2019-01-03,60.071999,35.547501
2019-01-04,63.537998,37.064999
2019-01-07,66.991997,36.982498
2019-01-08,67.070000,37.687500


In [18]:
# j'ajoute  le rendement de chaque actif a ma dataFrame

actifs['rdts_aapl'] = (actifs['AAPL'] -actifs['AAPL'].shift(1))/actifs['AAPL'].shift(1)

actifs['rdts_tsla'] = (actifs['TSLA'] -actifs['TSLA'].shift(1))/actifs['TSLA'].shift(1)
#
actifs.head()

Symbols,TSLA,AAPL,rdts_aapl,rdts_tsla,perfomances_prtf
Date,,,,,
2019-01-03,60.071999,35.547501,NaN,NaN,9344.604730
2019-01-04,63.537998,37.064999,0.042689,0.057697,10501.933636
2019-01-07,66.991997,36.982498,-0.002226,0.054361,10260.676584
2019-01-08,67.070000,37.687500,0.019063,0.001164,10101.137414
2019-01-09,67.706001,38.327499,0.016982,0.009483,10132.321977


In [19]:
# je supprime la première ligne car le rendement ne peut pas etre calculer sur cette dernière
actifs.dropna(inplace =True)

In [20]:
actifs.head()

Symbols,TSLA,AAPL,rdts_aapl,rdts_tsla,perfomances_prtf
Date,,,,,
2019-01-04,63.537998,37.064999,0.042689,0.057697,10501.933636
2019-01-07,66.991997,36.982498,-0.002226,0.054361,10260.676584
2019-01-08,67.070000,37.687500,0.019063,0.001164,10101.137414
2019-01-09,67.706001,38.327499,0.016982,0.009483,10132.321977
2019-01-10,68.994003,38.450001,0.003196,0.019023,10111.098143


In [78]:
#on transforme notre rendement journalier en valeur

prtf = 1000000

actifs['perfomances_prtf'] = prtf*( 0.5*actifs['rdts_aapl']+ 0.5*actifs['rdts_tsla'] )

actifs.head()

Symbols,TSLA,AAPL,rdts_aapl,rdts_tsla,perfomances_prtf
Date,,,,,
2019-01-04,63.537998,37.064999,0.042689,0.057697,50193.363634
2019-01-07,66.991997,36.982498,-0.002226,0.054361,26067.658360
2019-01-08,67.070000,37.687500,0.019063,0.001164,10113.741371
2019-01-09,67.706001,38.327499,0.016982,0.009483,13232.197737
2019-01-10,68.994003,38.450001,0.003196,0.019023,11109.814310


In [79]:
actifs[['TSLA','AAPL']].normalize().iplot(kind='lines')

In [80]:
# on affiche la perfomances de notre prtf

actifs[['perfomances_prtf']].cumsum().iplot(kind='line')


In [82]:
# on trie par odre décroissant et on calclu le centile à 5%

actif_tri= actifs['perfomances_prtf'].sort_values(ascending = False)
VaR = np.percentile( actif_tri, 5)

In [83]:
print('VaR un jour est : '+ str(-VaR))

VaR un jour est : 40341.77175990928


In [84]:
print(-VaR*np.sqrt(len(actif_tri)+1))

905670.5298060777


## 2) Méthode de Monté Carlo
